Inicialmente, após importação das dependências e constantes, os dados foram carregados e visualizados para um primeiro contato. Percebe-se que são 798 registros, com 12 características, onde dados faltantes, há escala diferente entre eles, e o problema é de classificação binária.

In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
DRE = 113080612

df = pd.read_csv('Lista1.csv.csv', index_col=0)
df

,Age,HealthFactor1,HealthFactor2,BMI,DiabetesPedigreeFunction,Pregnancies,Glucose,BloodPressure,PreviousSurgery,SkinThickness,Insulin,Inheritance,Outcome
Patient,,,,,,,,,,,,,
p1,50,46.72,176.0,33.6,627.000,6,148,72,0.0,35,0,NaN,1
p2,31,30.12,160.0,26.6,351.000,1,85,66,0.0,29,0,1.0,0
p3,32,30.26,88.0,23.3,672.000,8,183,64,0.0,0,0,1.0,1
p4,21,22.42,125.0,28.1,167.000,1,89,66,0.0,23,94,NaN,0
p5,33,35.02,NaN,43.1,2.288,0,137,40,0.0,35,168,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
p794,38,37.22,86.0,34.1,337.000,5,117,92,0.0,0,0,1.0,0
p795,60,55.20,117.0,36.0,546.000,5,109,75,0.0,26,0,NaN,0
p796,28,28.72,99.0,31.6,851.000,3,158,76,0.0,36,245,NaN,1


Após uma análise da variância das propriedades, percebemos que podemos eliminar o dado HealthFactor1, que possui altíssima correlação com Age.

In [2]:
corr = df.corr()
corr.style.background_gradient(cmap='coolwarm')

,Age,HealthFactor1,HealthFactor2,BMI,DiabetesPedigreeFunction,Pregnancies,Glucose,BloodPressure,PreviousSurgery,SkinThickness,Insulin,Inheritance,Outcome
Age,1.000000,0.986353,-0.011730,0.026066,-0.020133,0.540127,0.275171,0.240425,nan,-0.113536,-0.018745,-0.005511,0.238175
HealthFactor1,0.986353,1.000000,-0.033591,0.190297,-0.010066,0.531146,0.305851,0.278586,nan,-0.047240,0.012624,-0.009429,0.280172
HealthFactor2,-0.011730,-0.033591,1.000000,-0.134904,0.038598,0.014421,-0.021249,-0.064042,nan,-0.102973,-0.094756,-0.022908,-0.012707
BMI,0.026066,0.190297,-0.134904,1.000000,0.058930,0.004304,0.216251,0.257889,nan,0.390166,0.188423,-0.022819,0.280936
DiabetesPedigreeFunction,-0.020133,-0.010066,0.038598,0.058930,1.000000,-0.034349,0.022784,-0.008518,nan,0.107349,0.122489,-0.022449,0.093618
Pregnancies,0.540127,0.531146,0.014421,0.004304,-0.034349,1.000000,0.131023,0.140953,nan,-0.101265,-0.091575,0.019051,0.217109
Glucose,0.275171,0.305851,-0.021249,0.216251,0.022784,0.131023,1.000000,0.155191,nan,0.058374,0.342454,-0.030178,0.467702
BloodPressure,0.240425,0.278586,-0.064042,0.257889,-0.008518,0.140953,0.155191,1.000000,nan,0.195655,0.080238,0.001158,0.061076
PreviousSurgery,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
SkinThickness,-0.113536,-0.047240,-0.102973,0.390166,0.107349,-0.101265,0.058374,0.195655,nan,1.000000,0.437676,-0.129377,0.079457



Buscando os dados faltantes, obtemos 403 registros


In [3]:
rows_with_null_data = df[df.isnull().any(axis=1)]
rows_with_null_data

,Age,HealthFactor1,HealthFactor2,BMI,DiabetesPedigreeFunction,Pregnancies,Glucose,BloodPressure,PreviousSurgery,SkinThickness,Insulin,Inheritance,Outcome
Patient,,,,,,,,,,,,,
p1,50,46.72,176.0,33.6,627.000,6,148,72,0.0,35,0,NaN,1
p4,21,22.42,125.0,28.1,167.000,1,89,66,0.0,23,94,NaN,0
p5,33,35.02,NaN,43.1,2.288,0,137,40,0.0,35,168,0.0,1
p12,34,34.80,180.0,38.0,537.000,10,168,74,0.0,0,0,NaN,1
p13,57,51.02,57.0,27.1,1.441,10,139,80,0.0,0,0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
p790,41,39.02,180.0,31.1,205.000,10,125,70,NaN,26,115,NaN,1
p791,43,42.28,107.0,39.4,257.000,7,147,76,NaN,0,0,1.0,1
p795,60,55.20,117.0,36.0,546.000,5,109,75,0.0,26,0,NaN,0


Porém, analisando por colunas, percebemos que há uma concentração em certas colunas, enquanto a maioria das colunas estão completamente preenchidas. Optou-se portanto por remover a coluna Inheritance por completo, pois possuia ausência em metade dos dados disponíveis. Removemos também os registros faltantes de HealthFactor2, já que eram somente 11 registros.


In [4]:
df.isna().sum()

Age                           0
HealthFactor1                 0
HealthFactor2                11
BMI                           0
DiabetesPedigreeFunction      0
Pregnancies                   0
Glucose                       0
BloodPressure                 0
PreviousSurgery              46
SkinThickness                 0
Insulin                       0
Inheritance                 369
Outcome                       0
dtype: int64

Analisando os dados de PreviousSurgery, onde já se viu que há 46 dados faltantes, e o vemos que todos os outros registros têm valor 0. Optou-se por remover esta coluna. Finalmente, ficamos com 787 registros com 9 características.


In [5]:
df = df.drop(['Inheritance'], axis=1)
df = df.drop(['HealthFactor1'], axis=1)
df = df.dropna(subset=['HealthFactor2'])
df

,Age,HealthFactor2,BMI,DiabetesPedigreeFunction,Pregnancies,Glucose,BloodPressure,PreviousSurgery,SkinThickness,Insulin,Outcome
Patient,,,,,,,,,,,
p1,50,176.0,33.6,627.0,6,148,72,0.0,35,0,1
p2,31,160.0,26.6,351.0,1,85,66,0.0,29,0,0
p3,32,88.0,23.3,672.0,8,183,64,0.0,0,0,1
p4,21,125.0,28.1,167.0,1,89,66,0.0,23,94,0
p6,30,126.0,25.6,201.0,5,116,74,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
p794,38,86.0,34.1,337.0,5,117,92,0.0,0,0,0
p795,60,117.0,36.0,546.0,5,109,75,0.0,26,0,0
p796,28,99.0,31.6,851.0,3,158,76,0.0,36,245,1


In [6]:
df = df.drop(['PreviousSurgery'], axis=1)
df.groupby('PreviousSurgery').count()

,Age,HealthFactor2,BMI,DiabetesPedigreeFunction,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,Outcome
PreviousSurgery,,,,,,,,,,
0.0,741,741,741,741,741,741,741,741,741,741


In [7]:
df

,Age,HealthFactor2,BMI,DiabetesPedigreeFunction,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,Outcome
Patient,,,,,,,,,,
p1,50,176.0,33.6,627.0,6,148,72,35,0,1
p2,31,160.0,26.6,351.0,1,85,66,29,0,0
p3,32,88.0,23.3,672.0,8,183,64,0,0,1
p4,21,125.0,28.1,167.0,1,89,66,23,94,0
p6,30,126.0,25.6,201.0,5,116,74,0,0,0
...,...,...,...,...,...,...,...,...,...,...
p794,38,86.0,34.1,337.0,5,117,92,0,0,0
p795,60,117.0,36.0,546.0,5,109,75,26,0,0
p796,28,99.0,31.6,851.0,3,158,76,36,245,1


Performou-se em seguida o escalonamento dos dados


In [8]:
scaler = MinMaxScaler()
columns_to_scale = [
    'Age',
    'HealthFactor2',
    'BMI',
    'DiabetesPedigreeFunction',
    'Pregnancies',
    'Glucose',
    'BloodPressure',
    'SkinThickness',
    'Insulin'
]
df[columns_to_scale] = scaler.fit_transform(df[columns_to_scale])
df

,Age,HealthFactor2,BMI,DiabetesPedigreeFunction,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,Outcome
Patient,,,,,,,,,,
p1,0.483333,0.879397,0.500745,0.628849,0.352941,0.743719,0.590164,0.353535,0.000000,1
p2,0.166667,0.798995,0.396423,0.351991,0.058824,0.427136,0.540984,0.292929,0.000000,0
p3,0.183333,0.437186,0.347243,0.673989,0.470588,0.919598,0.524590,0.000000,0.000000,1
p4,0.000000,0.623116,0.418778,0.167419,0.058824,0.447236,0.540984,0.232323,0.111111,0
p6,0.150000,0.628141,0.381520,0.201525,0.294118,0.582915,0.606557,0.000000,0.000000,0
...,...,...,...,...,...,...,...,...,...,...
p794,0.283333,0.427136,0.508197,0.337948,0.294118,0.587940,0.754098,0.000000,0.000000,0
p795,0.650000,0.582915,0.536513,0.547598,0.294118,0.547739,0.614754,0.262626,0.000000,0
p796,0.116667,0.492462,0.470939,0.853546,0.176471,0.793970,0.622951,0.363636,0.289598,1


Para treinar o modelo, foram utilizados o GridSearchCV, que é uma classe do SKLearn para a otimização do modelo. Ele faz uma busca exaustiva pelos parãmetros fornecidos, criando modelos para cada combinação possível, e avaliando os modelos gerados. A avaliação é tada por validação cruzada de 5 partes (5 fold). 
É importante observar que estão sendo gerados modelos com diferentes arquiteturas de redes neurais. Mais especificamente, todas as arquiteturas:
 [5],
 [10],
 [20],
 [40],
 [5, 5],
 [10, 10],
 [20, 20],
 [40, 40],
 [5, 5, 5],
 [10, 10, 10],
 [20, 20, 20],
 [40, 40, 40],
 [5, 5, 5, 5],
 [10, 10, 10, 10],
 [20, 20, 20, 20],
 [40, 40, 40, 40]

In [11]:
df_x = df.drop(['Outcome'], axis=1)
df_y = df[['Outcome']]
x_train, x_test, y_train, y_test = train_test_split(df_x, df_y, test_size=0.30, random_state=DRE)
parameters = {
    'solver':('lbfgs', 'sgd', 'adam'),
    'activation': ('logistic', '‘tanh', 'relu'),
    'learning_rate': ('constant', 'invscaling', 'adaptive'),
    'hidden_layer_sizes': [ [i]*j for j in range(1, 5) for i in [5, 10, 20, 40] ],
    'random_state': [DRE],
}
clf = GridSearchCV(MLPClassifier(), parameters, cv=5, refit=True)
clf.fit(df_x, df_y)

GridSearchCV(cv=5, estimator=MLPClassifier(),
             param_grid={'activation': ('logistic', '‘tanh', 'relu'),
                         'hidden_layer_sizes': [[5], [10], [20], [40], [5, 5],
                                                [10, 10], [20, 20], [40, 40],
                                                [5, 5, 5], [10, 10, 10],
                                                [20, 20, 20], [40, 40, 40],
                                                [5, 5, 5, 5], [10, 10, 10, 10],
                                                [20, 20, 20, 20],
                                                [40, 40, 40, 40]],
                         'learning_rate': ('constant', 'invscaling',
                                           'adaptive'),
                         'random_state': [113080612],
                         'solver': ('lbfgs', 'sgd', 'adam')})

In [12]:
print(clf.best_estimator_)
print(clf.best_score_)

MLPClassifier(hidden_layer_sizes=[20, 20, 20, 20], random_state=113080612)
0.7663065387406274


O melhor modelo encontrado portanto, via validação cruzada 5-fold, foi o modelo MLPClassifier(hidden_layer_sizes=[20, 20, 20, 20], random_state=113080612), que obteve acurácia de aprox. 77%.